In [1]:
import os
import torch
from datasets import load_dataset, Dataset
import pandas as pd
import transformers
from transformers import AutoTokenizer
from trl import SFTTrainer
import transformers
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from pynvml import *
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import glob
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationSummaryMemory

In [2]:
directory_path = r'/home/sdm/Desktop/shakib/KUET LLM/data/'
file_pattern = '*.docx'  
file_paths = glob.glob(directory_path + file_pattern)
file_paths

['/home/sdm/Desktop/shakib/KUET LLM/data/rag4.docx',
 '/home/sdm/Desktop/shakib/KUET LLM/data/rag2V2.docx',
 '/home/sdm/Desktop/shakib/KUET LLM/data/rag1.docx']

In [3]:
from langchain import HuggingFacePipeline
# path='google/gemma-7b'
path="KUETLLM_zepyr"
tokenizer = AutoTokenizer.from_pretrained(path,
                                          use_auth_token=True,)



model = AutoModelForCausalLM.from_pretrained(
    path,
    # quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,

)
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )
    
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0})


/home/sdm/anaconda3/envs/llmtesting/lib/python3.9/site-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:

def create_vectorstore(flag):
    if flag == False:
        flag=True
        # loader = Docx2txtLoader(r"data/Khulna University of Engineering.docx")
        # documents=loader.load()
        all_doc=[]
        for x in file_paths:
            loader = Docx2txtLoader(x)
            documents=loader.load()
            all_doc.extend(documents)
        # text_splitter=splitter_ret()
        text_splitter=CharacterTextSplitter(chunk_size=500,chunk_overlap=30,separator="\n")
        docs=text_splitter.split_documents(documents=all_doc)
        # text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        # docs = text_splitter.split_documents(documents)
        model_name = "sentence-transformers/all-mpnet-base-v2"
        hf = HuggingFaceEmbeddings(model_name=model_name)
        vectorstore=FAISS.from_documents(docs,hf)
        vectorstore.save_local('vectorstore')
        return vectorstore
    else:
        model_name = "sentence-transformers/all-mpnet-base-v2"
        hf = HuggingFaceEmbeddings(model_name=model_name)
        new_vectorstore=FAISS.load_local("vectorstore",hf)
        return new_vectorstore

In [5]:
from langchain_core.prompts import ChatPromptTemplate
template='''
You are an assistant for question-answering tasks. Use the following pieces of retrieved context and history to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question}
history: {history} 
Context: {context} 
Answer:
'''
prompt = ChatPromptTemplate.from_template(template)

In [6]:
retriever = create_vectorstore(True).as_retriever()

/home/sdm/anaconda3/envs/llmtesting/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
from langchain.chains import (
    StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
)
combine_docs_chain = StuffDocumentsChain(
    num_docs=5,  # Number of documents to consider
    similarity_threshold=0.7,  # Threshold for document relevance
    summarization_method='extractive',  # Method for summarizing documents
    language='en'  # Language of the documents
)
question_generator_chain = LLMChain(llm=llm, prompt=prompt)
chain = ConversationalRetrievalChain(
    combine_docs_chain=combine_docs_chain,
    retriever=retriever,
    question_generator=question_generator_chain,
)

KeyError: 'llm_chain'

In [ ]:
import gradio as gr
questions=[
    'How many halls in KUET?', 'weather of KUET',"What is the most beautiful things in KUET", "IS KUET RAGGING FREE?","Why KUET CSE is called CSE Family? "
]
demo = gr.ChatInterface(fn=ans_ret, examples=questions, title="KUET information Bot")
demo.launch(share=False)